## BCH Code

In [4]:
%run RS.ipynb

class BCHCode:
    
    def __init__(self, n, b, D):
        
        self.q = 2
        self.n = n
        self.m = Mod(self.q,self.n).multiplicative_order()
        self.b = b
        self.D = D
        self.C_RS = RSCode(self.n, self.n - self.D + 1, self.q**self.m)
            
        if (self.n != self.q**self.m - 1):
            raise ValueError('Invalid input values: n != q^m - 1')
            
        self.message_type = 'bin'
        
        # Initialize field
        self.F = GF(self.q) # base field
        self.EF = GF(self.q**self.m) # extension field
        R.<x> = PolynomialRing(self.F, 'x')
        self.R = R
        self.x = x
        self.alpha = self.EF.primitive_element()
        
        # Constructing generator matrix
        self.cosets = self.cyclotomic_cosets(self.n, self.q, self.b, self.D)
        
        self.generator_poly = self.BCH_generator_polynomial(self.x, self.alpha, self.D, self.cosets)
        
        if not (self.generator_poly.divides(self.x**self.n - 1)):
            raise ValueError('generator_poly is not a generator polynomial')
            
        self.k = self.n - self.generator_poly.degree()
        
        self.G = matrix(self.F, self.k, self.n, lambda i,j : self.generator_poly[(j+(self.n-i)) % self.n])
        
        
    def cyclotomic_cosets(self, n, q, b, D):
        # compute cyclotomic cosets
    
        cosets = []

        for i in range(b,b+D-1):
            coset = [(i * q**j) % n for j in range(0,n-1)]
            coset = list(set(coset))
            coset.sort()
            cosets.append(coset)
        return cosets

    def minimal_polynomial(self, coset, x, alpha):
        # compute minimal polynomial from one coset
        poly = 1
        for j in range(len(coset)):
            poly *= (x - alpha**coset[j])
        return poly

    def BCH_generator_polynomial(self, x, alpha, D, cosets):
        # compute generator polynomial
        poly = self.minimal_polynomial(cosets[0], x, alpha)
        for i in range(1,D-1):
            poly = LCM(poly,self.minimal_polynomial(cosets[i],x,alpha))
        
        return poly
    
    
    def Encoding(self, message, zeropad = True, out = 'bin', concatenated_dimension = None):
        
        if not concatenated_dimension:
            concatenated_dimension = self.k
        else:
            concatenated_dimension = concatenated_dimension
        
        data_type = self._DetermineInput(message)
        
        if (data_type == 'pol' or data_type == 'bin'):
            message = list(message)
        elif data_type == 'int':
            pass
        else:
            raise ValueError('Wrong data type')
            
        
        rem = len(message) % concatenated_dimension
        
        if rem != 0:
            if zeropad:
                message.extend([self.F(0)]*(concatenated_dimension-rem))
            else:
                raise ValueError('k does not divide input size')      
                

        c = []
        
        # Encoding each chunk of size k
        for i in range(0, len(message), self.k):
            c.extend(self.EncodeChunk(message[i:i+self.k]))
        
        if out == 'pol':
            return vector(self.F, c)
        elif out == 'int':
            c = self._PolToInt(c)
            return c
        elif out == 'bin':
            c = self._PolToInt(c)
            c = self._IntToBitString(c)
            return c
        else:
            raise ValueError('Unrecognized output')
            
            
    def EncodeChunk(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.F, chunk) * self.G
        
        return c
                
    
    def Decoding(self, received, out = 'bin'):
        
        data_type = self._DetermineInput(received)
        
        if data_type == 'pol':
            received = vector(self.F, received)
        elif data_type == 'bin':
            received = vector(self.F, received)
        elif data_type == 'int':
            received = vector(self.F, received)
        else:
            raise ValueError('Wrong data type')
        
        # Check input size
        if len(received) % (self.n) != 0:
            raise ValueError('Invalid input size')
            
        d = []
        
        for i in range(0, len(received), self.n):
            d.extend(self.DecodeChunk(received[i:i+self.n]))
            
        if out == 'pol':
            return vector(self.F, d)
        elif out == 'int':
            d = self._PolToInt(d)
            return d
        elif out == 'bin':
            d = self._PolToInt(d)
            d = self._IntToBitString(d)
            return d
        else:
            raise ValueError('Unrecognized output')
            
    
    def DecodeChunk(self, chunk):
        
        if (len(chunk) != self.n):
            raise ValueError('Invalid input size')
        
        # Need to convert to extension field
        for i in range(len(chunk)):
            chunk[i] = self.EF(chunk[i])
          
        # Decode with RS decoder
        chunk = self.C_RS.DecodeChunk(chunk)
        
        chunk = self.C_RS.EncodeChunk(chunk)
        
        # Need to convert to base field
        for i in range(len(chunk)):
            chunk[i] = self.F(chunk[i])
        
        cols = self.G.pivots()
        G_independent = self.G.matrix_from_columns(cols)
        
        c = self.DecodeChunkBCH(chunk, cols, G_independent)
            
        return c
    
    def DecodeChunkBCH(self, chunk, cols, G_independent):
        
        chunk_independent = [chunk[i] for i in cols]
        
        return vector(self.F, chunk_independent) * G_independent.inverse()
    
    
    def _DetermineInput(self, data):
        # determine data type

        if isinstance(data, str) and all([(bit == '1' or bit == '0') for bit in data]):

            return('bin')

        if data[0] == None:
            return('none')
        elif data[0].parent() == self.F:
            return('pol')
        elif data[0].parent() == ZZ:
            return('int')
        else:
            return('unknown')

    def _PolToInt(self, pol_array):

        # convert array of polynomial representation to array of integers

        pol_out = []

        for pol in pol_array:
            if not pol in self.F:
                raise ValueError('Invalid symbol')

            pol_out.append(ZZ(pol.polynomial().coefficients(sparse = False), base = self.q))

        return(pol_out)
    
    def _IntToBitString(self, int_array):

        # Converts array of integers less than 2 to bit string

        if any([(item > (self.q - 1) or item < 0) for item in int_array]):
            raise ValueError('Invalid integer values')

        return(''.join([format(item, '01b') for item in int_array]))

In [5]:
#C = BCHCode(n = 15, b=1, D = 7) 
#C.k

In [6]:
#m = vector(GF(2), [1,1,0,0,1,1,0,0,1,1,0,0,1,1,0,0,1])
#m = [1,1,0,0,1,1,0,0,1,1,0,0,1,1,0,0,1]
#m = '11001100110011001'

#c = C.Encoding(m, out = 'pol')
#c = C.Encoding(m, out = 'int')
#c = C.Encoding(m, out = 'bin')
#print('codeword: ', c)

#d = C.Decoding(c, out = 'pol')
#d = C.Decoding(c, out = 'int')
#d = C.Decoding(c, out = 'bin')
#print('decoded word: ', d)


It is now shown that the product code can correct $\delta=\lfloor\frac{d_{BCH}d_{REP}-1}{2}\rfloor$ errors:

In [7]:
#C = BCHREPCode(n_BCH = 1023, b = 1, D = 5, n_REP = 31)

#m = []
#for i in range(C.code_k):
#    m.append(C.BF.random_element())
    

#print('delta = ', C.code_delta)
    
#c = C.Encoding(m)

#positions = []
#for i in range(C.code_delta): # add delta errors
#    position = ZZ.random_element(0,C.code_n)
#    while position in positions:
#        position = ZZ.random_element(0,C.code_n)  
      
#    positions.append(position)
#    c[position] = C.BF(c[position] - 1) # flip the bit

#d = C.Decoding(c)

#print('Decoding status: ', d == m)